In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import glob
import calendar
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import SelectFromModel

pd.options.display.max_columns = None
pd.options.display.max_rows = None

warnings.filterwarnings('ignore')

os.chdir("E:/Trinity/Dissertation/Sem 3/models/data/")

In [6]:
os.chdir("E:/Trinity/Dissertation/Sem 3/models/")
df = pd.read_csv("HourlyDetails (Unboosted).csv")  

In [8]:
df["ScaledStartTime"] = df["StartTime"]
df["ScaledDayofWeek"] = df["DayOfWeek"]

In [11]:
df.head()

,Date,StartTime,EndTime,No_of_Cars,DayOfWeek,Weekend?,isHoliday?,Month,Day,Year,Tolls,Tollname_Dublin Tunnel,Tollname_Eastlink,Tollname_Limerick Tunnel,Tollname_M1,Tollname_M3,Tollname_M4,Tollname_M50,Tollname_M7-M8,Tollname_M8,Tollname_N25,Tollname_N6,ScaledStartTime,ScaledDayofWeek
0,01-01-2019,0,1,22,2,0,1,-1.552307,-1.676356,2019,M7-M8,0,0,0,0,0,0,0,1,0,0,0,-1.661325,-0.974781
1,01-01-2019,1,2,29,2,0,1,-1.552307,-1.676356,2019,M7-M8,0,0,0,0,0,0,0,1,0,0,0,-1.516862,-0.974781
2,01-01-2019,2,3,25,2,0,1,-1.552307,-1.676356,2019,M7-M8,0,0,0,0,0,0,0,1,0,0,0,-1.372399,-0.974781
3,01-01-2019,3,4,31,2,0,1,-1.552307,-1.676356,2019,M7-M8,0,0,0,0,0,0,0,1,0,0,0,-1.227936,-0.974781
4,01-01-2019,4,5,24,2,0,1,-1.552307,-1.676356,2019,M7-M8,0,0,0,0,0,0,0,1,0,0,0,-1.083473,-0.974781


In [10]:
cols_to_norm = ['ScaledStartTime','ScaledDayofWeek']
df[cols_to_norm] = StandardScaler().fit_transform(df[cols_to_norm])

In [57]:
df["TrafficCategory"] = 0

In [12]:
print(df["No_of_Cars"].min())
print(df["No_of_Cars"].max())

0
4282


In [22]:
tolls = df["Tolls"].unique()
for toll in tolls:
    print(toll + "------", end= " ")
    print(df[df["Tolls"] == toll]["No_of_Cars"].max())
    print(df[df["Tolls"] == toll]["No_of_Cars"].mean())
    #print(df[df["Tolls"] == toll]["No_of_Cars"].min())
    print()

M7-M8------ 1502
487.93266574585635

N6------ 821
251.5084706959707

N25------ 763
172.7964876033058

M50------ 4282
1597.181392694064

M1------ 2756
877.7171052631579

M8------ 947
242.52314814814815

M3------ 3056
808.9490870786517

Eastlink------ 1419
463.0063131313131

M4------ 1874
568.8199404761905

-------- 3
0.10818713450292397

Dublin Tunnel------ 1092
212.05363720073666

Limerick Tunnel------ 1881
231.10802902621722



In [74]:
tolls = df["Tolls"].unique()
for toll in tolls:
    print(toll)
    quants = df[df["Tolls"] == toll]["No_of_Cars"].quantile([0.25,0.5,0.75])
    print(quants)
    df.loc[(df["Tolls"]==toll) & (df["No_of_Cars"]<=quants[0.25]),"TrafficCategory"] = 1
    df.loc[(df["Tolls"]==toll) & (df["No_of_Cars"]>quants[0.25]) & (df["No_of_Cars"]<=quants[0.50]),"TrafficCategory"] = 2
    df.loc[(df["Tolls"]==toll) & (df["No_of_Cars"]>quants[0.50]) & (df["No_of_Cars"]<=quants[0.75]),"TrafficCategory"] = 3
    df.loc[(df["Tolls"]==toll) & (df["No_of_Cars"]>quants[0.75]),"TrafficCategory"] = 4

M7-M8
0.25    100.0
0.50    499.0
0.75    800.0
Name: No_of_Cars, dtype: float64
N6
0.25     40.0
0.50    248.0
0.75    421.0
Name: No_of_Cars, dtype: float64
N25
0.25     16.0
0.50    129.0
0.75    300.0
Name: No_of_Cars, dtype: float64
M50
0.25     274.0
0.50    1252.5
0.75    3014.0
Name: No_of_Cars, dtype: float64
M1
0.25     122.00
0.50     758.50
0.75    1420.25
Name: No_of_Cars, dtype: float64
M8
0.25      0.0
0.50     78.0
0.75    491.0
Name: No_of_Cars, dtype: float64
M3
0.25      82.0
0.50     637.5
0.75    1256.0
Name: No_of_Cars, dtype: float64
Eastlink
0.25     57.0
0.50    417.0
0.75    839.0
Name: No_of_Cars, dtype: float64
M4
0.25      76.0
0.50     445.5
0.75    1008.0
Name: No_of_Cars, dtype: float64
--
0.25    0.0
0.50    0.0
0.75    0.0
Name: No_of_Cars, dtype: float64
Dublin Tunnel
0.25      0.0
0.50     80.0
0.75    367.0
Name: No_of_Cars, dtype: float64
Limerick Tunnel
0.25      0.0
0.50     37.0
0.75    398.0
Name: No_of_Cars, dtype: float64


In [77]:
tolls = df["Tolls"].unique()
for toll in tolls:
    print(toll)
    print(df[df["Tolls"]==toll]["TrafficCategory"].value_counts())

M7-M8
1    2176
3    2173
2    2170
4    2169
Name: TrafficCategory, dtype: int64
N6
1    2234
3    2198
4    2169
2    2135
Name: TrafficCategory, dtype: int64
N25
1    2215
3    2179
4    2166
2    2152
Name: TrafficCategory, dtype: int64
M50
1    2193
3    2191
4    2189
2    2187
Name: TrafficCategory, dtype: int64
M1
1    2054
4    2052
3    2052
2    2050
Name: TrafficCategory, dtype: int64
M8
1    1814
3    1672
4    1669
2    1541
Name: TrafficCategory, dtype: int64
M3
3    2139
1    2138
2    2134
4    2133
Name: TrafficCategory, dtype: int64
Eastlink
1    2192
4    2177
2    2172
3    2171
Name: TrafficCategory, dtype: int64
M4
1    2204
3    2187
4    2181
2    2164
Name: TrafficCategory, dtype: int64
--
1    1244
4     124
Name: TrafficCategory, dtype: int64
Dublin Tunnel
1    2620
4    2171
3    2162
2    1735
Name: TrafficCategory, dtype: int64
Limerick Tunnel
1    2892
4    2133
3    2118
2    1401
Name: TrafficCategory, dtype: int64


In [99]:
df.head()

,Date,StartTime,EndTime,No_of_Cars,DayOfWeek,Weekend?,isHoliday?,Month,Day,Year,Tolls,Tollname_Dublin Tunnel,Tollname_Eastlink,Tollname_Limerick Tunnel,Tollname_M1,Tollname_M3,Tollname_M4,Tollname_M50,Tollname_M7-M8,Tollname_M8,Tollname_N25,Tollname_N6,ScaledStartTime,ScaledDayofWeek,TrafficCategory
0,01-01-2019,0,1,22,2,0,1,-1.552307,-1.676356,2019,M7-M8,0,0,0,0,0,0,0,1,0,0,0,-1.661325,-0.974781,1
1,01-01-2019,1,2,29,2,0,1,-1.552307,-1.676356,2019,M7-M8,0,0,0,0,0,0,0,1,0,0,0,-1.516862,-0.974781,1
2,01-01-2019,2,3,25,2,0,1,-1.552307,-1.676356,2019,M7-M8,0,0,0,0,0,0,0,1,0,0,0,-1.372399,-0.974781,1
3,01-01-2019,3,4,31,2,0,1,-1.552307,-1.676356,2019,M7-M8,0,0,0,0,0,0,0,1,0,0,0,-1.227936,-0.974781,1
4,01-01-2019,4,5,24,2,0,1,-1.552307,-1.676356,2019,M7-M8,0,0,0,0,0,0,0,1,0,0,0,-1.083473,-0.974781,1


In [100]:
df.to_csv("MoreCategoriesData.csv", index=False)

In [102]:
df["TrafficCategory"].value_counts()

1    25976
4    23333
3    23242
2    21841
Name: TrafficCategory, dtype: int64